In [15]:
import scipy.io.wavfile as wv
import os
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np
import scipy.signal as ss

In [16]:
def folders (FoldersList):
    for fold in FoldersList:
        #check is folder exists, if not - create
        if (os.path.isdir(fold)==0):
            os.mkdir(fold)

In [17]:
def signal_interval(SampleRate, data_f, step, step_coef, file):
    over_std = np.where(abs(data_f)>data_f.std())[0]
    N, = over_std.shape
    
    over_std_cont = np.zeros ((len(data_f),1))
    n = 0
    while (n<N-step):
        if over_std[n+step]-over_std[n]<=step*step_coef: #where is no gap
            over_std_cont[over_std[n]:over_std[n+step]]=1
        n+=1
    diff = over_std_cont[:-1]-over_std_cont[1:]
    starts = np.where (diff==-1)[0] 
    ends = np.where (diff==1)[0]
    
    if len(starts)<len(ends):
        starts.insert(0, 0) #first start will be the beginnibg of the trace
    elif len(starts)>len(ends):
        ends.insert(len(data), -1) #last start is end of trace
    durations = ends-starts
    start_p = starts[durations.argmax()]
    end_p = ends[durations.argmax()]
    
    ###plotting###
    plt.figure()
    for n in range(N):
        plt.axvline(over_std[n])
    plt.plot(data_f, 'k')
    plt.axvline(start_p, color = 'r')
    plt.axvline(end_p, color = 'r')
    plt.savefig(os.path.join(path, 'Signal_Interval', file+'.png'))
    plt.close('all')
    
    return start_p, end_p

In [18]:
def to_png(SampleRate, data, file):
    #filtering
    f_cutoff = [500, 3200]
    Wn = [f_cutoff[0] / (SampleRate/2) , f_cutoff[1] / (SampleRate/2)]
    b, a = ss.iirfilter(1, Wn, ftype='butter', btype='bandpass')
    data_f = ss.filtfilt(b, a, data)

    #start and end of signal
    start_p, end_p = signal_interval(SampleRate, data_f, 10, 80, file)
    
    ###resampling###
    Resampling = 10000
    data_res = ss.resample(data_f[start_p:end_p], Resampling)
    SampleRate_res = Resampling/(end_p-start_p)*SampleRate

    #spectrogramm###
    plt.subplot(1,2,1)
    f, t, Sxx = ss.spectrogram(data_res, fs = SampleRate_res)
    plt.pcolormesh(t, f, Sxx, cmap=cm.gray)
    plt.xlim(t[0],t[-1])
    plt.ylim(500,3200)
    plt.yscale('log')
    plt.axis('off')
    plt.margins(0.0)

    ###trace###
    plt.subplot(1,2,2)
    plt.plot(data_res, 'k')
    plt.axis('off')
    plt.margins(0.0)
    plt.savefig(os.path.join(path, Destin_Dir, file+'.png'), bbox_inches="tight", cmap='gray')
    plt.close('all')

In [22]:
path = os.getcwd()
WAV_Dir = 'numbers_test'
filelist = os.listdir(os.path.join(path,WAV_Dir))
Destin_Dir = 'TestSamples'
Signal_Interval_Folder = 'Signal_Interval'
folders ([Destin_Dir, Signal_Interval_Folder])

for file in filelist:
    SampleRate,data = wv.read(os.path.join(path, WAV_Dir, file))
    to_png(SampleRate, data, file)